# CbmRoot Notebook

This notebook provides code snippets how one can run simulations & analysis in a notebook, employing the CbmRoot classes.

In [1]:
# When using the CbmRootPy kernel, this also loads CbmRoot classes
import ROOT
ROOT.gROOT.SetBatch(True)
ROOT.PyConfig.DisableRootLogon = True

import os


Welcome to JupyROOT 6.26/10


## The transport macro

Here we have the macro from `macro/run` just with python syntax. Note that all CBM related classes are available in the `ROOT` package/namespace.

First we define tha macro as function, then we call it in the next cell.

In [2]:
# /* Copyright (C) 2020 GSI Helmholtzzentrum fuer Schwerionenforschung, Darmstadt
#    SPDX-License-Identifier: GPL-3.0-only
#    Authors: Volker Friese [original file committer], Ralf Kliemt */
ROOT.gROOT.Reset()

input = ""
nEvents = 1
output = "test"  
setup = "sis100_electron"
randomSeed = 0
overwrite = True

# --- Logger settings ----------------------------------------------------
ROOT.FairLogger.GetLogger().SetLogScreenLevel("INFO")
ROOT.FairLogger.GetLogger().SetLogVerbosityLevel("LOW")

# -----   Environment   --------------------------------------------------
myName = "CbmJupyTransport"                 # this macro's name for screen output
srcDir = ROOT.gSystem.Getenv("VMCWORKDIR")  # top source directory

# -----   Input file   ---------------------------------------------------
# Use default input if not specified by the user
defaultInput = srcDir + "/input/urqmd.auau.10gev.centr.root"
inFile = input if input != "" else defaultInput
print(f"-I- {myName}: Using input file {inFile}")

# -----   Output files   -------------------------------------------------
dataset = output
outFile = dataset + ".tra.root"
parFile = dataset + ".par.root"
geoFile = dataset + ".geo.root"
if overwrite:
  if os.path.exists(outFile):
    os.remove(outFile)
    print(f"The file {outFile} has been deleted.")
  if os.path.exists(parFile):
    os.remove(parFile)
    print(f"The file {parFile} has been deleted.")
  if os.path.exists(geoFile):
    os.remove(geoFile)
    print(f"The file {geoFile} has been deleted.")

# -----   Target properties   --------------------------------------------
targetMaterial  = "Gold"
targetThickness = 0.025  # in cm
targetDiameter  = 2.5    # in cm
targetZpos = -44.0
# The target is at -44 cm from the centre of the magnet.
print(f"Target is at {targetZpos}cm from origin")

# -----   Beam properties   ----------------------------------------------
beamPosX   = 0.
beamPosY   = 0.
beamSigmaX = 0.1  # cm
beamSigmaY = 0.1  # cm
# -----   Event plane rotation   -----------------------------------------
rotateEvents = True

# --- Transport run preparation-------------------------------------------
print("Creating a RUN object")
timer = ROOT.TStopwatch()
timer.Start()
run = ROOT.CbmTransport()
engine = ROOT.kGeant3
run.SetEngine(engine)
if "pluto" in inFile.lower():
  run.AddInput(inFile, ROOT.kPluto)
else:
  run.AddInput(inFile, ROOT.kUnigen)
run.SetOutFileName(outFile, overwrite)
run.SetParFileName(parFile)
run.SetGeoFileName(geoFile)

print("Loading the setup")
run.LoadSetup(setup)
if not run.GetSetup().IsActive(ROOT.ECbmModuleId.kTarget):
  print("Target being generated in tra macro")
  run.SetTarget(targetMaterial, targetThickness, targetDiameter, 0, 0, targetZpos)
run.SetBeamPosition(beamPosX, beamPosY, beamSigmaX, beamSigmaY)
if rotateEvents: 
  run.SetRandomEventPlane()
run.SetRandomSeed(randomSeed)

# --- Transport run execution -------------------------------------------
print(f"Starting the RUN with {nEvents} events")
run.Run(nEvents)

# ------------------------------------------------------------------------
# -----   Finish   -------------------------------------------------------
timer.Stop()
rtime = timer.RealTime()
ctime = timer.CpuTime()
sysInfo = ROOT.FairSystemInfo()
maxMemory = sysInfo.GetMaxMemory()
cpuUsage = ctime / rtime
print(f"Macro finished successfully.")
print(f"Real time {rtime} s, CPU time {ctime}s")
print(f"MaxMemory {maxMemory}")
print(f"CpuLoad{cpuUsage}")
ROOT.gROOT.Reset()
# ------------------------------------------------------------------------
# End of macro


-I- CbmJupyTransport: Using input file /u/kliemt/CBM/CbmRoot/Install/share/cbmroot/input/urqmd.auau.10gev.centr.root
The file test.tra.root has been deleted.
The file test.par.root has been deleted.
The file test.geo.root has been deleted.
Target is at -44.0cm from origin
Creating a RUN object
Loading the setup
Target being generated in tra macro
Starting the RUN with 1 events
Macro finished successfully.
Real time 13.448765993118286 s, CPU time 13.010000000000002s
MaxMemory 861.9609375
CpuLoad0.9673749998072091
[INFO] File /u/kliemt/CBM/CbmRoot/Install/share/cbmroot/input/urqmd.auau.10gev.centr.root is a ROOT file.
[INFO] UnigenGenerator: Initialising...
[INFO] UnigenGenerator: Mode  standard; rotate event plane to zero
[INFO] UnigenGenerator: Open input file /u/kliemt/CBM/CbmRoot/Install/share/cbmroot/input/urqmd.auau.10gev.centr.root
[INFO] UnigenGenerator: Run description
--------------------------------------------------
-I-                 Run Header                 -I-
Generator

Info in <TGeoManager::TGeoManager>: Geometry FAIRGeom, FAIR geometry created
Info in <TGeoManager::SetTopVolume>: Top volume is cave. Master volume is cave
Info in <TGeoNavigator::BuildCache>: --- Maximum geometry depth set to 100
Warning in <TGeoMaterial::ctor>: Material MAG_Vacuum defined with fractional Z=0.500000
Info in <TGeoManager::CheckGeometry>: Fixing runtime shapes...
Info in <TGeoManager::CheckGeometry>: ...Nothing to fix
Info in <TGeoManager::CloseGeometry>: Counting nodes...
Info in <TGeoManager::Voxelize>: Voxelizing...
Info in <TGeoManager::CloseGeometry>: Building cache...
Info in <TGeoManager::CountLevels>: max level = 10, max placements = 372
Info in <TGeoManager::CloseGeometry>: 577444 nodes/ 1092 volume UID's in FAIR geometry
Info in <TGeoManager::CloseGeometry>: ----------------modeler ready----------------
Error in <TExMap::Remove>: key 166863080 not found at 3052
Error in <TExMap::Remove>: key 166863280 not found at 3252
Error in <TExMap::Remove>: key 166863472 

## A quick plot

Plotting with ROOT is still working. To show a canvas use its `Draw()` function.

In [3]:
ROOT.gROOT.Reset()
%jsroot on
asimfile = ROOT.TFile("test.tra.root")
atree = asimfile.Get("cbmsim")
canvas = ROOT.TCanvas()
atree.Draw("StsPoint.fZ","")
canvas.Draw()
